# Numba CUDA intro

## Short summary CUDA components

1. host: CPU
2. device: GPU
3. host memory: system main memory
4. device memory: onboard memory on a GPU card
5. kernel: a GPU function launched by the host and executed on the device
6. device function: a GPU function executed on the device which can only be called from the device (i.e. from a kernel or another device function)

![](http://upload.wikimedia.org/wikipedia/commons/thumb/5/59/CUDA_processing_flow_%28En%29.PNG/450px-CUDA_processing_flow_%28En%29.PNG)

### CUDA structure

A grid contains blocks with threads:

![](https://www.researchgate.net/profile/Omar-Bouattane/publication/321666991/figure/fig2/AS:572931245260800@1513608861931/Figure-2-Execution-model-of-a-CUDA-program-on-NVidias-GPU-Hierarchy-grid-blocks-and.png)

## Use CUDA with Numba

**NOTE**: If you don't have a CUDA capable GPU, you can activate the _simulation_ mode by starting Jupyterlab with the `NUMBA_ENABLE_CUDASIM=1` ENV:

> `NUMBA_ENABLE_CUDASIM=1 jupyter-lab`

In [2]:
from numba import cuda
print(cuda.gpus) # -> Will probably not work, if you don't have a GPU

<Managed Device 0>


## Kernels

> GPU function called form the CPU

- Pass and get data through arrays
- Before calling Kernel function, declare thread hierarchy

In [3]:
# Our jit cuda enabled function 
@cuda.jit
def test_kernel(an_array):
    """
    TODO Code for kernel.
    """


In [4]:
import numpy as np

# Declare kernel:

# 1. Data array to work with
# Docs: "Return a new array of given shape and type, filled with ones."
dataArr = np.ones(256)

# 2. Define Thread values

# Block size depending on size data array, shared memory, supported hardware, ...
threads_in_block = 32
blocks_in_grid = (dataArr.size + (threads_in_block - 1))

# 3. "Call" Kernel
test_kernel[blocks_in_grid, threads_in_block](dataArr)

print(dataArr)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


### Positon of thread in grid & block

> Get the position of thread by getting the information in the kernel

In [5]:
@cuda.jit
def position_kernel(an_array):
    # Let's get the thread and corresponding block
    tt = cuda.threadIdx.x # Aka. X-Dimension
    tb = cuda.blockIdx.x # Aka. Y-Dimension
    
    # "Size" aka. width of Block: Number threads in Block
    bs = cuda.blockDim.x
    
    position = tt + tb * bs

    if position < an_array.size:
        an_array[position] *= 2 # Just double size as example
    

In [6]:
import numpy as np

# Same as above:
dataArr = np.ones(256)
threads_in_block = 32
blocks_in_grid = (dataArr.size + (threads_in_block - 1))

position_kernel[blocks_in_grid, threads_in_block](dataArr)

# TODO understand
print(dataArr)

[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]


### Automating position search

> Automate search for position using `numba`

In [8]:
@cuda.jit
def automated_kernel(an_array):
    # Docs: "Return the absolute position of the current thread in the entire grid of blocks."
    # `ndim` => Number dimensions
    position = cuda.grid(1)
        
    # Same as above
    if position < an_array.size:
        an_array[position] *= 2 
    

Add host to call Kernel:

In [9]:
import numpy as np

# Same as above:
dataArr = np.ones(256)
threads_in_block = 32
blocks_in_grid = (dataArr.size + (threads_in_block - 1))

automated_kernel[blocks_in_grid, threads_in_block](dataArr)

print(dataArr)

[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]


## Memory management

> Use manual memory management to have more efficient code

Example using matrix multiplication:

See: https://en.wikipedia.org/wiki/Matrix_multiplication


In [8]:
@cuda.jit
def matmul(inputArr1, inputArr2, outputArr):
    """
    Do a Matrix multiplication: inputArr1 * inputArr2 = outputArr
    """
    # Get a two dimensional grid for calculations
    row, column = cuda.grid(2)
    
    # Check that we're in the boundaries
    # & not accessing prohibited memory
    if row < outputArr.shape[0] and column < outputArr.shape[1]:
        tmp = 0.
        for k in range(inputArr1.shape[1]):
            tmp += inputArr1[row, k] * inputArr2[k, column]
            outputArr[row, column] = tmp

In [9]:
import math

# Create 2D-Arrays filled with 7 and 8s
# NOTE: Row <-> Column or Column <-> Row should be equal size
inputArr1 = np.full((12, 42), 7, float)
inputArr2 = np.full((42, 16), 8, float)

# Copy 2D-Arrays to device aka "GPU"
inputArr1_global_mem  = cuda.to_device(inputArr1)
inputArr2_global_mem  = cuda.to_device(inputArr2)

# Allocate mem on device for result
# Shape = non-equal size values from above
outputArr_global_mem = cuda.device_array((12,16))

# TODO how get values about threadsperblock?
threads_in_block = (16, 16)
blocks_per_grid_x = int(math.ceil(inputArr1.shape[0] / threads_in_block[0]))
blocks_per_grid_y = int(math.ceil(inputArr1.shape[1] / threads_in_block[1]))
blocks_in_grid = (blocks_per_grid_x, blocks_per_grid_y)

matmul[blocks_in_grid, threads_in_block](inputArr1_global_mem, inputArr2_global_mem, outputArr_global_mem)

# Copy result back to host aka. CPU
outputArr = outputArr_global_mem.copy_to_host()

print(outputArr)





[[2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352.
  2352. 2352. 2352. 2352.]
 [2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352.
  2352. 2352. 2352. 2352.]
 [2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352.
  2352. 2352. 2352. 2352.]
 [2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352.
  2352. 2352. 2352. 2352.]
 [2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352.
  2352. 2352. 2352. 2352.]
 [2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352.
  2352. 2352. 2352. 2352.]
 [2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352.
  2352. 2352. 2352. 2352.]
 [2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352.
  2352. 2352. 2352. 2352.]
 [2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352.
  2352. 2352. 2352. 2352.]
 [2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352. 2352.
  2352. 2352. 235